In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
repository = '/content/drive/MyDrive/MyDocuments/Academic/Education after grad/UCB/IEOR 2019/Academics/Capstone/Project/Git/DemandPredictionJD/'
# data_directory = 'capstone_jd_msom_data-minus-click-data/'
%cd {repository} 
%ls -a

/content/drive/MyDrive/MyDocuments/Academic/Education after grad/UCB/IEOR 2019/Academics/Capstone/Project/Git/DemandPredictionJD
 capstone_jd_msom_data-minus-click-data/
'Clustering with K-prototype .ipynb'
 .git/
 initial_data_cleaning_order.ipynb
 Inventory_optimization.ipynb
 .ipynb_checkpoints/
 order_after_engineer.csv
 order_aggregation_Lei.ipynb
 README.md
 sku_clustering.csv
 sku_with_cluster.csv
 warehouse_capacity_preprocessing.ipynb


In [ ]:
'''
x → contextual data
SKU, brand, activate date, order date, discount, type ..
d → unknown parameter that depends on x
Demand
w → decision variable
Inventory quantity for the month
c → cost vector
Cost of allocating w products in the distribution center
c = h + s
h: inventory holding cost of not selling off all products
s: delivery cost due to transporting products to meet demands

min{c(w, d) = h(w - d) + s(d - w)}
'''

In [17]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155113 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [22]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.w = Var(domain=NonNegativeReals)    # decision variable. inventory order quantity
model.d = Var(domain=NonNegativeReals)    # demand
model.inv_cost = Var(domain=NonNegativeReals)
model.ship_cost = Var(domain=NonNegativeReals)

# declare objective
# inventory cost = 10*max(model.w - model.d, 0)
# shipping cost = 30*max(model.d - model.w, 0)
model.cost = Objective(expr = model.inv_cost + model.ship_cost, sense=minimize)

# declare constraints
model.actual_demand = Constraint(expr = model.d == 100)
model.capacity_limit = Constraint(expr = model.w <= 1000)
model.inventory_cost = Constraint(expr = model.inv_cost >= 10*(model.w - model.d))
model.shipping_cost = Constraint(expr = model.ship_cost >= 30*(model.d - model.w))

model.pprint()

4 Var Declarations
    d : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    inv_cost : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    ship_cost : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    w : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    cost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : inv_cost + ship_cost

4 Constraint Declarations
    actual_demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None : 100.0 :    d

In [23]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# display solution
print('\nCost = ', model.cost())

print('\nDecision Variables')
print('w = ', model.w())
print('d = ', model.d())
print('inv_cost = ', model.inv_cost())
print('ship_cost = ', model.ship_cost())

print('\nConstraints')
print('Actual Demand  = ', model.actual_demand())
print('Capacity Limit  = ', model.capacity_limit())
print('Inventory Cost = ', model.inventory_cost())
print('Shipping Cost = ', model.shipping_cost())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 5
  Number of nonzeros: 9
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.016257762908935547
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------